In [1]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")

c:\Users\mihir\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\mihir\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\mihir\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\mihir\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Set up dataset and parameters

In [2]:
# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

buildings_df = buildings_df.sample(n=20000, random_state=1)  # Remove later

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)

In [3]:
# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])

# Create arrays to track ordering (grocery stores)
grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
grocery_stores = grocery_stores.sort_values('building_id')
grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

grocery_stores_array = np.array(grocery_stores['building_id'])
grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])

In [4]:
# Create parameter matrices (Res comm access matrix - Bij)
# [i,j] value indicates whether residential building i is within access distance of commercial building j
res_comm_distance_matrix, res_comm_access_matrix = dc.calculate_access(res_buildings_coordinates_array, comm_buildings_coordinates_array)

# Save file
np.save('res_comm_access_matrix', res_comm_access_matrix)
np.save('res_comm_distance_matrix', res_comm_distance_matrix)

In [5]:
# Load the files and use it 
res_comm_access_matrix = np.load('res_comm_access_matrix.npy')
res_comm_distance_matrix = np.load('res_comm_access_matrix.npy')

In [6]:
# Create parameter matrices (Res groc access array - Aj)
# ith value indicates whether the ith residential building has existing access
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)
res_access_array = np.amax(res_groc_access_matrix, 1)

In [7]:
# Create parameter matrices (Res Population - Pj)
# ith value indicates the population in the ith column
res_population = pa.get_population(geopandas_dataframe=res_buildings) 
res_population_array = np.array(res_population['population'])
res_population_array

array([3, 1, 2, ..., 1, 1, 1], dtype=int64)

In [8]:
# Create demand matrix - number of unsatisfied customers
res_demand = res_population_array
res_demand[np.where(res_access_array == 1)] = 0 # setting effective demand to zero for buildings that have existing access to a grocery store

### Optimization

In [9]:
# Figure out thresholding later

# np.where(res_comm_distance_matrix <= 1)

# res_comm_distance_matrix <= 1


# pairings = {(c, r): res_comm_distance_matrix
#             for facility in range(num_candidates)
#             for cluster in range(num_clusters) 
#             if  dist(facility_locs[facility], centroids[cluster]) < threshold}
# print("Number of viable pairings: {0}".format(len(pairings.keys())))

In [10]:
### Testing modeling stuff

# Parameters
num_commercial_buildings = res_comm_distance_matrix.shape[1]
num_residential_buildings = len(res_demand)
num_stores = 1

# Implement model
m = gp.Model('Facility location')

# Decision variables 
select = m.addVars(range(num_commercial_buildings), vtype=GRB.BINARY, name='select') # select location
assign = m.addVars(range(num_residential_buildings), range(num_commercial_buildings), vtype=GRB.BINARY, name='assign') # assignment of residential building to cluster

# Objective function - min total distance from residential buildings to their assigned grocery store, multiplied by demand
m.setObjective(sum(sum(res_demand[i] * res_comm_distance_matrix[i,j] * assign[i, j] for i in range(num_residential_buildings)) for j in range(num_commercial_buildings)))
m.modelSense = GRB.MINIMIZE

# Constraints
m.addConstr(sum(select[i] for i in range(len(select))) <= num_stores, name='store_limit')

#m.addConstr(select.sum() <= num_stores, name = 'store_limit') # Facility limit
for i in range(num_residential_buildings):
    m.addConstr(sum(assign[i,j] for j in range(num_commercial_buildings)) ==  1) # can only assign each residential building to one store

for i in range(num_residential_buildings):
    for j in range(num_commercial_buildings):
        m.addConstr(assign[i,j] <= select[j], name='open2assign') # locations can only be assigned demand if they are selected

# Optimize
m.optimize()


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-24
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 21011215 rows, 20993448 columns and 62978120 nonzeros
Model fingerprint: 0x3a6d0179
Variable types: 0 continuous, 20993448 integer (20993448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 0 columns (presolve time = 8s) ...
Presolve removed 0 rows and 0 columns (presolve time = 10s) ...
Presolve removed 0 rows and 0 columns (presolve time = 18s) ...
Presolve removed 0 rows and 0 columns (presolve time = 21s) ...
Presolve removed 0 rows and 0 columns (presolve time = 27s) ...
Presolve removed 0 rows and 0 columns (presolve time = 33s) ...
Presolve removed 0 rows and 0 columns (presolve time = 36s) ...


In [101]:
m.objVal

1018.0862875957736

In [102]:
counter = 0
for i in range(len(select)):
    counter += select[i].x

counter

3.0